In [59]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, fbeta_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from skmultilearn.adapt import MLkNN
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier 
     
from nltk.stem.snowball import SnowballStemmer

import pandas as pd
import json
import os
import re

import gensim 
from gensim.models import Word2Vec
import gensim.downloader
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pickle 

import nltk
from nltk.tokenize import word_tokenize

import spacy
import matplotlib
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
from ast import literal_eval
from tqdm import tqdm

import sklearn.metrics
import numpy as np

import pickle

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
import flair
from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings

from sklearn.model_selection import GridSearchCV

import sys

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /homes/lgf21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
tqdm.pandas()

# Opening Files: 

In [4]:
# Open Pickle without lemmatisation: 

with open('merged_data_no_duplicates.pickle', 'rb') as handle:
    (X_train_text, X_test_text, Y_train, Y_test, _, _) = pickle.load(handle)

In [6]:
import json
print(json.dumps({'sentence' :X_train_text.iloc[1]}, indent=4))

{
    "sentence": "Emergency Incident ResponseReport a Confirmed or Potential Breach? Call +1 770-870-6343 Blog Contact Support Login Secureworks Products Solutions Services Why Secureworks Insights Company Research The Curious Case of Mia Ash: Fake Persona Lures Middle Eastern Targets THREAT ANALYSIS The Curious Case of Mia Ash: Fake Persona Lures Middle Eastern Targets SecureWorks\\xc2\\xae Counter Threat Unit\\xe2\\x84\\xa2 Threat Intelligence THURSDAY, JULY 27, 2017 BY: COUNTER THREAT UNIT RESEARCH TEAM Summary In early 2017, SecureWorks\\xc2\\xae Counter Threat Unit\\xe2\\x84\\xa2 (CTU) researchers observed phishing campaigns targeting several entities in the Middle East and North Africa (MENA), with a focus on Saudi Arabian organizations. The campaigns delivered PupyRAT, an open-source cross-platform remote access trojan (RAT). CTU\\xe2\\x84\\xa2 researchers observed likely unsuccessful phishing campaigns being followed by highly targeted spearphishing and social engineering atta

In [5]:
# Open Pickle lemmatised: 

#with open('merged_data_lemma.pickle', 'rb') as handle:
#    (X_train_text, X_test_text, Y_train, Y_test, _, _) = pickle.load(handle)

# Feature Extraction:

In [5]:
sys.path.append('../src')

In [6]:
# call method
from methods import feature_extraction, evaluation

## Count Vectorizer:

In [60]:
#X_train, X_test = feature_extraction('CountVectorizer', X_train_text, X_test_text, fe_filename = '/homes/lgf21/API/app/CV_tactic.pickle')

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## TF-IDF:

In [74]:
X_train, X_test = feature_extraction('TfIdfVectorizer', X_train_text, X_test_text)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## word2vec Google news:

In [14]:
#w2v_google = gensim.downloader.load('word2vec-google-news-300')

In [15]:
#X_train, X_test = feature_extraction('embedding', X_train_text, X_test_text, embedding_type = w2v_google, weighted=False)

2072it [01:05, 31.56it/s] 
441it [00:08, 50.83it/s]


## Glove:

In [ ]:
#glv = gensim.downloader.load('glove-wiki-gigaword-100')

In [9]:
#X_train, X_test = feature_extraction('embedding', X_train_text, X_test_text, embedding_type = glv)

2072it [04:47,  7.21it/s]
441it [00:44,  9.95it/s]


## Trained word2vec:

In [43]:
#w2v = Word2Vec.load("word2vec.model").wv

In [44]:
#X_train, X_test = feature_extraction("embedding", X_train_text, X_test_text, embedding_type = w2v, weighted=False)

2072it [01:31, 22.68it/s]
441it [00:12, 35.53it/s]


## Trained Doc2Vec:

In [8]:
#doc2vec = Doc2Vec.load("doc2vec.model")

In [9]:
#X_train, X_test = feature_extraction("embedding", X_train_text, X_test_text, embedding_type = doc2vec, weighted=False)

2072it [13:08,  2.63it/s]
441it [01:09,  6.36it/s]


In [26]:
doc2vec.infer_vector([])

array([ 3.8924557e-03,  2.2277397e-03,  2.8260357e-03,  3.1031757e-03,
        2.4691331e-03, -3.8241588e-03, -3.1321845e-03,  1.9334984e-03,
        4.6540769e-03, -2.8597424e-03,  2.4144650e-03,  1.7765344e-03,
        3.7589096e-03, -4.2618732e-03, -4.9383338e-03, -1.3230800e-03,
       -4.4076736e-03,  4.2729466e-03,  3.7056184e-03,  2.0966656e-03,
        1.4571905e-04,  1.4837455e-03,  3.4156006e-03, -3.3563317e-04,
       -6.8738940e-04, -1.0160482e-03,  3.2573866e-03, -4.6786680e-03,
        3.6644626e-03, -2.6009798e-03, -1.3250709e-04,  4.6174917e-03,
       -2.6511133e-04, -7.1959796e-05,  6.5771700e-04, -1.3252646e-04,
        4.6653245e-03,  2.8656316e-03,  4.7203214e-03,  2.3470731e-03,
        4.3927552e-03, -1.2031364e-03,  2.1725125e-03,  3.1157995e-03,
       -3.7453657e-03,  2.8423613e-03,  1.1108667e-03,  4.5181038e-03,
        2.5888907e-03, -4.2246254e-03, -4.7815233e-03, -1.1290354e-03,
       -4.9470365e-04, -7.6360523e-04, -3.8795057e-03, -5.1941094e-04,
      

# Classifiers:

## Naive Bayes:

In [15]:
naive_bayes_classifier = OneVsRestClassifier(MultinomialNB())
naive_bayes_classifier.fit(X_train, Y_train)

OneVsRestClassifier(estimator=MultinomialNB())

In [16]:
y_pred_proba = pd.DataFrame(naive_bayes_classifier.predict_proba(X_test), columns = Y_test.columns)

In [17]:
y_pred = (y_pred_proba > 0.005).astype(int) # if increase threshold, recall decreases and precision (could) increase

In [18]:
evaluation(y_pred, Y_test)

,metric,result
0,macro precision,0.664535
1,micro precision,0.407468
2,macro recall,0.254589
3,micro recall,0.205233
4,macro fscore,0.336271
5,micro fscore,0.340385


## Decision Tree:

In [75]:
dt = OneVsRestClassifier(DecisionTreeClassifier(random_state=0))
dt.fit(X_train, Y_train)

OneVsRestClassifier(estimator=DecisionTreeClassifier(random_state=0))

In [76]:
Y_pred = pd.DataFrame(dt.predict(X_test), columns=Y_test.columns)

In [77]:
evaluation(Y_pred, Y_test)

,metric,result
0,macro precision,0.506236
1,micro precision,0.547203
2,macro recall,0.480061
3,micro recall,0.511856
4,macro fscore,0.499328
5,micro fscore,0.539748


## SVC:

In [45]:
# Train and test: First delete techniques less than 9 
# We fix the random state to have the same dataset in our different tests

sv_classifier = OneVsRestClassifier(LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42), n_jobs = 1)
sv_classifier.fit(X_train, Y_train)


/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced', dual=False,
                                        random_state=42),
                    n_jobs=1)

In [46]:
Y_pred = pd.DataFrame(sv_classifier.predict(X_test), columns=Y_test.columns)

In [47]:
evaluation(Y_pred, Y_test)

,metric,result
0,macro precision,0.419702
1,micro precision,0.450093
2,macro recall,0.586692
3,micro recall,0.593622
4,macro fscore,0.437239
5,micro fscore,0.472964


## Multi-label KNN: 

In [12]:
knn = MLkNN(k = 3)

In [ ]:
# only works old version sklearn 

In [13]:
# train
knn.fit(X_train.values, Y_train.values)

# predict
predictions = knn.predict(X_test.values)

AttributeError: 'list' object has no attribute 'values'

In [ ]:
evaluation(predictions, Y_test)

## Logistic Regression:

In [52]:
# reduce dimension using pca: 

pca = PCA(n_components=75)
pca.fit(X_train)
pca_result = pca.transform(X_train)
x_test_result = pca.transform(X_test )

In [53]:
log_reg = OneVsRestClassifier(LogisticRegression(random_state=0, multi_class='multinomial', solver='lbfgs', max_iter = 1000)).fit(X_train, Y_train)

predictions = log_reg.predict(X_test)


/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

In [54]:
evaluation(predictions, Y_test)

,metric,result
0,macro precision,0.552238
1,micro precision,0.585980
2,macro recall,0.362440
3,micro recall,0.437449
4,macro fscore,0.482542
5,micro fscore,0.548718


## DT AdaBoost: 

In [81]:
dt_adaboost = OneVsRestClassifier(AdaBoostClassifier(n_estimators=100, random_state=0)).fit(X_train, Y_train)
predictions_ada = dt_adaboost.predict(X_test)

In [69]:
evaluation(predictions_ada, Y_test)

,metric,result
0,macro precision,0.651022
1,micro precision,0.665706
2,macro recall,0.533804
3,micro recall,0.566639
4,macro fscore,0.619091
5,micro fscore,0.643215


## Random Forest:

In [29]:
predictions_rf = OneVsRestClassifier(RandomForestClassifier(max_depth=2, random_state=0)).fit(X_train, Y_train)

In [30]:
evaluation(predictions_rf, Y_test)

TypeError: Expected sequence or array-like, got <class 'sklearn.multiclass.OneVsRestClassifier'>

# Classifer Chain: 

In [70]:
def chain_model(model):
    model_chain = ClassifierChain(model, order='random', random_state=0)
    return model_chain

In [82]:
chain = chain_model(dt_adaboost) # change model appropriately

In [ ]:
chainModel = chain.fit(X_train, Y_train)
predictions = chainModel.predict(X_test)

In [ ]:
evaluation(predictions, Y_test)

# Neural Networks:

In [59]:
model = GridSearchCV(MLPClassifier(random_state=1), 
                     {'hidden_layer_sizes': [[100,100], [100, 100, 1000], [200, 200], [1000, 200], [300, 300], [300, 200]],'learning_rate':['adaptive', 'constant']},
                    scoring='f1_macro')

model.fit(X_train, Y_train)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(estimator=MLPClassifier(random_state=1),
             param_grid={'hidden_layer_sizes': [[100, 100], [100, 100, 1000],
                                                [200, 200], [1000, 200],
                                                [300, 300], [300, 200]],
                         'learning_rate': ['adaptive', 'constant']},
             scoring='f1_macro')

In [60]:
pd.DataFrame(model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,183.839936,43.541096,0.254782,0.171545,"[100, 100]",adaptive,"{'hidden_layer_sizes': [100, 100], 'learning_r...",0.609884,0.612155,0.561256,0.561503,0.581334,0.585227,0.022297,9
1,165.466637,17.413085,0.229404,0.220633,"[100, 100]",constant,"{'hidden_layer_sizes': [100, 100], 'learning_r...",0.609884,0.612155,0.561256,0.561503,0.581334,0.585227,0.022297,9
2,137.560856,22.566673,0.335453,0.176796,"[100, 100, 1000]",adaptive,"{'hidden_layer_sizes': [100, 100, 1000], 'lear...",0.567722,0.600417,0.537735,0.568746,0.584168,0.571757,0.020776,11
3,137.986200,19.995427,0.141784,0.167322,"[100, 100, 1000]",constant,"{'hidden_layer_sizes': [100, 100, 1000], 'lear...",0.567722,0.600417,0.537735,0.568746,0.584168,0.571757,0.020776,11
4,276.353142,80.568188,0.236424,0.195722,"[200, 200]",adaptive,"{'hidden_layer_sizes': [200, 200], 'learning_r...",0.613156,0.616244,0.558272,0.573154,0.583445,0.588854,0.022591,3
5,268.783451,58.207433,0.227457,0.191253,"[200, 200]",constant,"{'hidden_layer_sizes': [200, 200], 'learning_r...",0.613156,0.616244,0.558272,0.573154,0.583445,0.588854,0.022591,3
6,1148.548388,357.905833,0.850722,0.208619,"[1000, 200]",adaptive,"{'hidden_layer_sizes': [1000, 200], 'learning_...",0.615574,0.620034,0.554765,0.585671,0.580508,0.591310,0.024074,1
7,1557.160791,466.211931,1.054398,0.408399,"[1000, 200]",constant,"{'hidden_layer_sizes': [1000, 200], 'learning_...",0.615574,0.620034,0.554765,0.585671,0.580508,0.591310,0.024074,1
8,543.426751,171.339277,0.745838,0.500287,"[300, 300]",adaptive,"{'hidden_layer_sizes': [300, 300], 'learning_r...",0.608380,0.620157,0.556832,0.578976,0.576750,0.588219,0.022929,5
9,335.581551,45.820021,0.375709,0.146973,"[300, 300]",constant,"{'hidden_layer_sizes': [300, 300], 'learning_r...",0.608380,0.620157,0.556832,0.578976,0.576750,0.588219,0.022929,5


In [24]:
mlp = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes = [1000, 200]).fit(X_train, Y_train)

In [25]:
predictions_mlp = mlp.predict(X_test)

In [26]:
evaluation(predictions_mlp, Y_test)

,metric,result
0,macro precision,0.766092
1,micro precision,0.764583
2,macro recall,0.561157
3,micro recall,0.600164
4,macro fscore,0.704776
5,micro fscore,0.724867


## Multi Layer Perceptron: 

In [12]:
mlp = MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes = [100]).fit(X_train, Y_train)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [13]:
predictions_mlp = mlp.predict(X_test)

In [14]:
evaluation(predictions_mlp, Y_test)

,metric,result
0,macro precision,0.766950
1,micro precision,0.751969
2,macro recall,0.577053
3,micro recall,0.624693
4,macro fscore,0.708780
5,micro fscore,0.722527


## Loading data from flair:

In [27]:
with open('fasttext_format_test.txt', 'w') as file:
    for i in range(len(Y_test)):
        file.write(' '.join(['__label__'+ col for col in Y_test.columns if Y_test.iloc[i][col] == 1]) + ' ' + X_test_text.iloc[i] + '\n')

In [28]:

flair.device = 'cpu'
# this is the folder in which train, test and dev files reside
data_folder = '.'

# load corpus containing training, test and dev data
corpus = ClassificationCorpus(data_folder,
                                      test_file='fasttext_format_test.txt',
                                      dev_file='fasttext_format_test.txt',
                                      train_file='fasttext_format_train.txt',
                                      label_type='tactic',
                                      )

2022-08-16 14:03:32,395 Reading data from .
2022-08-16 14:03:32,404 Train: fasttext_format_train.txt
2022-08-16 14:03:32,405 Dev: fasttext_format_test.txt
2022-08-16 14:03:32,406 Test: fasttext_format_test.txt
2022-08-16 14:03:34,091 Initialized corpus . (label type name is 'tactic')


In [29]:
# label to be predicted: 
label_type = 'tactic'
# create the label dictionary
label_dict = corpus.make_label_dictionary(label_type=label_type)

2022-08-16 14:03:34,104 Computing label dictionary. Progress:


2152it [02:33, 14.02it/s]

2022-08-16 14:06:08,192 Dictionary created for label 'tactic' with 13 values: TA0005 (seen 1237 times), TA0003 (seen 857 times), TA0002 (seen 756 times), TA0004 (seen 742 times), TA0011 (seen 688 times), TA0007 (seen 659 times), TA0006 (seen 487 times), TA0009 (seen 465 times), TA0008 (seen 312 times), TA0001 (seen 248 times), TA0010 (seen 199 times), TA0040 (seen 190 times)


## Transformers:

In [1]:
# initialize transformer document embeddings (many models are available)
document_embeddings = TransformerDocumentEmbeddings('binay1999/text_classification_cybertexts', fine_tune=True)

# create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, label_type=label_type, multi_label=True)

# initialize trainer
trainer = ModelTrainer(classifier, corpus)

# run training with fine-tuning
trainer.fine_tune('test_model',
                  learning_rate=5.0e-4,
                  mini_batch_size=30,
                  max_epochs=5,
                  )

NameError: name 'TransformerDocumentEmbeddings' is not defined

## LSTM (with Glove):

In [45]:
embedding = WordEmbeddings('en-glove')
document_embeddings = DocumentRNNEmbeddings([embedding])

In [ ]:
# create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, label_type=label_type, multi_label=True)

# initialize trainer
trainer = ModelTrainer(classifier, corpus)

# run training with fine-tuning
trainer.fine_tune('test_model_word2vec',
                  learning_rate=5.0e-4,
                  mini_batch_size=30,
                  max_epochs=3,
                  )

2022-08-14 12:23:08,455 ----------------------------------------------------------------------------------------------------
2022-08-14 12:23:08,462 Model: "TextClassifier(
  (decoder): Linear(in_features=128, out_features=13, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): BCEWithLogitsLoss()
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'en-glove'
        (embedding): Embedding(400001, 100)
      )
    )
    (word_reprojection_map): Linear(in_features=100, out_features=100, bias=True)
    (rnn): GRU(100, 128, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (weights): None
  (weight_tensor) None
)"
2022-08-14 12:23:08,466 ----------------------------------------------------------------------------------------------------
2022-08-14 12:23:08,469 Corpus: "Corpus: 2152 train + 4